In [ ]:
import keras
keras.__version__

# Generación de textos con una LSTM

Vamos a implementar una LSTM en Keras. Lo primero que necesitamos es gran cantidad de texto para poder aprender un modelo de lingüistica. Se puede usar cualquier archivo grande de texto. En este ejemplo vamos a usar El Principito (de Antoine de Saint-Exupery) que no es un libro muy extenso. Nuestro modelo aprenderá así un modelo específico basado en el estilo de escritura de Saint-Exupery en ese libro concreto. 


## Preparando los datos

Lo primero que hacemos es descargar el corpus y pasarlo todo a minúsculas.

In [ ]:
import keras
import numpy as np

path = keras.utils.get_file(
    'principito.txt',
    origin='http://bdigital.bnjm.cu/docs/libros/PROC2-2443/PROCE4383/El%20Principito.txt')
text = open(path).read().lower()
print('Longitud del corpus:', len(text))


A continuación extraeremos frases que tengan un solapamiento parcial de longitud `maxlon`, las convertiremos en un vector one-hot, y las meteremos en un array 3D de Numpy `x` cuya estructura corresponda a `(n_frases, maxlon, caracteres_unicos)`. Simultaneamente prepararemos un array `y` que contenga los targets correspondientes: los vectores one-hot de los caracteres que vienen justo después de cada frase extraida. 

In [ ]:
# Length of extracted character sequences
maxlon = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlon, step):
    sentences.append(text[i: i + maxlon])
    next_chars.append(text[i + maxlon])
print('Número de frases:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Caracteres únicos:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorización...')
x = np.zeros((len(sentences), maxlon, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

## Construyendo la red

Nuestra red no es mas que una única capa `LSTM` seguida por un clasificador `Denso` y un softmax sobre todos los posibles caracteres. 

In [ ]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlon, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

Como nuestros targets son vectores one-hot, usaremos `categorical_crossentropy` como función de pérdida de nuestro modelo:

In [ ]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## Entrenando el modelo lingüistico y sampleando a partir de él


Dado un modelo entrenado y un fragmento de texto como semilla, generaremos un nuevo texto siguiendo reiteradamente estos pasos: 

*  Extraer a partir del modelo la distribución de probabilidad para el texto dado hasta ese momento.
*  Repesar la distribución para una cierta "temperatura"
*  Samplear el siguiente caracter aleatoriamente de acuerdo a la distribución repesada
*  Añadir ese caracter al final del texto disponible.

Con este codigo repesamos la probabilidad original que viene del modelo y extraemos un indice de caracteres (función de "sampleo"):

In [ ]:
def sample(preds, temperatura=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperatura
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Finalmente, aqui tenemos el bucle en el entrenaremos y generaremos el texto. 


In [ ]:
import random
import sys

for epoch in range(1, 20):
    print('Época: ', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlon - 1)
    generated_text = text[start_index: start_index + maxlon]
    print('--- Generando con la siguiente semilla: "' + generated_text + '"')

    for temperatura in [0.3]:
        print('------ Temperatura:', temperatura)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlon, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperatura)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


## Tareas

*  Utiliza tu propio corpus en lugar de El Principito (puede ser en otros idiomas)
*  Modifica el bucle para que recorra varias temperaturas a cada vez (entre 0.1 y 1 por ejemplo), de modo que podamos ir comparando para cada época que aspecto tiene el texto resultante dependiendo de dicha temperatura.
*  Entrena para 60 epocas.
*  ¿Qué observas en el texto según vas variando la temperatura? ¿Cuál te parece la temperatura óptima y por qué?










